In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
import scipy
import glob
import matplotlib.pyplot as plt
import tqdm as tqdm
import os
import warnings
warnings.filterwarnings('ignore')
import sys

In [2]:
def df_to_sparse_matrix(df):
    df.columns=['i','j','p']
    df.i = df.i-1
    df.j = df.j-1
    new = df.pivot(index='i', columns='j', values='p')
    return csr_matrix((df.p, (df.i, df.j)))

In [3]:
df_train = pd.read_csv('/scratch/lemercier/WIP_data/train_data.csv')

In [4]:
unique_seq = df_train.sequence_id.unique()

In [5]:
dg = pd.read_csv('/scratch/lemercier/WIP_data/Ribonanza_bpp_files/extra_data/all_files.txt')

In [6]:
dg.columns=['name']
dg['parse_name'] = dg.name.apply(lambda x: x.split(' ')[-1][:31])
dg['new_name'] = dg.name.apply(lambda x: x.split(' ')[-1][31:])
dg = dg.query('parse_name=="Ribonanza_bpp_files/extra_data/"')
dg = dg[['new_name']]
dg['sequence_id'] = dg['new_name'].apply(lambda x: x.split('/')[-1][:-4])
dg['folder0'] = dg['new_name'].apply(lambda x: x.split('/')[0])
dg['folder1'] = dg['new_name'].apply(lambda x: x.split('/')[1])
dg['folder2'] = dg['new_name'].apply(lambda x: x.split('/')[2])
dg['full_folder']=dg.new_name.apply(lambda x: x[:6])
dg['in_train'] = dg.isin({'sequence_id': unique_seq}).sequence_id

**get all folders that contain train sequences**

In [7]:
dg_train = dg.query('in_train==True')
unique_train =  np.unique(dg_train.full_folder.values)

**unzip these folders one by one, compute and save the matrices, remove the files unziped**

In [8]:
base = '/scratch/lemercier/WIP_data/Ribonanza_bpp_files/extra_data/'
zip_file = base+'stanford-ribonanza-rna-folding.zip'
tmp = base+'tmp'

for folder in tqdm.tqdm(unique_train):
    
    # create tmp folder where to unzip
    os.system("mkdir {0}".format(tmp))
              
    # unzip the folder
    os.system("unzip {0} 'Ribonanza_bpp_files/extra_data/{1}*' -d {2} > /dev/null".format(zip_file, folder, tmp))
    
    # location of unziped files
    location = base+'tmp/Ribonanza_bpp_files/extra_data/'+folder
    
    # get all file names at this location 
    all_files = glob.glob(location+'*.txt') 
    
    # compute and store the matries
    for file in all_files:
        sequence_id = file.split('/')[-1]
        df = pd.read_csv(location+sequence_id, ' ', header=None)
        B = df_to_sparse_matrix(df)
        scipy.sparse.save_npz(base+'train/'+sequence_id[:-4]+'.npz', B)
    
    # remove unziped files 
    os.system("rm -r {}".format(tmp))

100%|█████████████████████████████████████████| 347/347 [30:43<00:00,  5.31s/it]
